In [1]:
import importlib
import xarray as xr
import numpy as np
import pandas as pd

from CASutils import filter_utils as filt
from CASutils import calendar_utils as cal
import sys

importlib.reload(filt)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

In [4]:
data = xr.open_dataset("/project/cas/islas/python_savs/GHCND/T2m_GHCND_NH_1979_2014.nc")
t2m = data['t2m']
lon = data['lon']
lat = data['lat']

In [5]:
# calculate the seasonal cycle for each station and subtract to produce anomalies
t2mseason = t2m.groupby('time.dayofyear').mean('time', skipna=True)
t2m4harm = filt.calc_season_nharm(t2mseason, 4, dimtime=1)
anoms = t2m.groupby('time.dayofyear')-t2m4harm

you got nans


In [6]:
# group by djfs and subtract the DJF mean
djfanoms = cal.group_season_daily(anoms,'DJF')
djfmean = djfanoms.mean('day', skipna=True)
djfanoms = djfanoms-djfmean

moving time axis to the start
nyears=35.0


In [7]:
ghcndvar = np.var(djfanoms, axis=(0,1))
ghcndvar = ghcndvar.rename('ghcndvar')

/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/core/nputils.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [8]:
ghcndvar.to_netcdf("/project/cas/islas/python_savs/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_GHCND.nc")
lon.to_netcdf("/project/cas/islas/python_savs/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_GHCND.nc", mode="a")
lat.to_netcdf("/project/cas/islas/python_savs/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_GHCND.nc", mode="a")

In [9]:
print(ghcndvar)

<xarray.DataArray 'ghcndvar' (station: 2765)>
array([ 3.71327526, 22.3222041 ,  8.63061739, ...,  1.86131017,
        2.68949974,  0.68820718])
Coordinates:
  * station  (station) object 'AE000041196' 'AFM00040938' ... 'WQW00041606'
